<img src="pexels-afta-putta-gunawan-683039.jpg" alt="coffee shop">

# Coffee Meet Data

Our client, a fictional coffee chain in New York, has asked us to take a look at their records. They want insight into any important factors that may help their business. They provided a selection of sales, product, customer, and other business information.

We'll use a combination of **Pandas** and **SQL** to pull out any releveant information. Then we'll chart this information using popular interactive visualization library **Plotly Express**.

## Dataset

There are nine tables of sample retail data provided:

* Sales Receipts
* Pastry Inventory
* Sales Targets
* Customer
* Dates
* Product
* Sales Outlet
* Staff
* Generation

Source: [IBM/Kaggle](https://www.kaggle.com/datasets/ylchang/coffee-shop-sample-data-1113)


## Potential Insights


1. Busiest Time of Day
2. Best selling products
3. Staff Contribution Ranking
4. Registered Customer Conversion Rate
5. Best Customer
5. Customer Generational Makeup
6. Online vs Instore

# Data Dictionary - Customers

|Column Name| Description|
|-----------|-----------|
|**customer_id**|A unique id assigned upon registration to each customer|
|**home_store**|The store a customer was registered at or has set as their main location|
|**customer_first-name**|Customers first name and last name|
|**customer_email**|Customers email used for registration|
|**customer_since**|How long a customer has been registered or when they first started shopping.|
|**loyalty_card_number**|A unique id used for a loyalty rewards program|
|**birthdate**|Customers Date of Birth - YYYY-MM-DD|
|**gender**|Customers registered gender|
|**birth_year**|Customers Year of Birth |

# Data Dictionary - Sales

|Column Name| Description|
|-----------|-----------|
|**transaction_id**|A unique id assigned to each purchase.|
|**transaction_date**|Calendar date when a transaction was made YYYY-MM-DD|
|**transaction_time**|Time when a transaction was made HH-MM-SS|
|**sales_outlet_id**|A unique used to identify which store handled the sale|
|**staff_id**|Unique id assigned to each staff member. Based on which staff member processed the transaction|
|**customer_id**|A unique id assigned upon registration to each customer|
|**instore_yn**|Was this purchase made in store or online|
|**order**|Unspecified|
|**line_item_id**|Unspecified|
|**product_id**|A unique id assigned to each product|
|**quantity**|The amount of each product purchased|
|**line_item_amount**|Unspecified|
|**unit_price**|Unit price for the product specified|
|**promo_item_yn**|Was the product a part of a promotional campaign|


## Library Imports

In [1]:
# data 
import pandas as pd
import matplotlib as plt
import plotly.express as px 
import datetime as dt

# sql 
import sqlite3

#system
from pathlib import Path

In [2]:
%%HTML
<style>
td {
  font-size: 18px
}
</style>

## Database Connection

In a real world scenario your dataset isn't alway contained in a handy cleaned csv file, but in a production database.

Here we create and populate a sqlite3 database in order to simulate real world conditions.

We use pandas to convert our sql queries into a dataframe for analysis.

In [3]:
# create our initial db file
Path('coffee.db').touch()

# create a db connection
connection = sqlite3.connect("coffee.db")

# cursor - database iterator
c = connection.cursor()

# create a sample table to verify
# c.execute('''CREATE TABLE sample_table (u_id int, email text)''')

# load the data into a dataframe
customers = pd.read_csv("customer.csv")

# write df to a sqlite table
customers.to_sql('customers', connection, if_exists="append", index=False) # 2246 results

# data for our second table/df
sales = pd.read_csv("2019_04_sales_reciepts.csv")

# second table
sales.to_sql('sales', connection, if_exists='append', index=False) # 49894 results

# verify
# fetchall returns an array of tuples
# c.execute('''SELECT *  FROM customers''').fetchall()

# verify second table
# c.execute('''SELECT *  FROM sales''').fetchall()

# join our two tables
c.execute('''SELECT * FROM sales s LEFT JOIN customers c ON c.customer_id = s.customer_id''')
# c.fetchall()

## Query to Dataframe conversion

Once we've pulled the data we need from our database, it needs to be converted into dataframe for ease of analysis.

The customer_id column was duplicated after our conversion, so we can safely drop it using a ~ (bitwise negation operator) and a call to col.duplicated().

In [4]:
# import our joined sql tables into a dataframe
coffee = pd.read_sql('''SELECT * FROM sales s LEFT JOIN customers c ON c.customer_id = s.customer_id''', connection)

# delete duplicate column
coffee = coffee.loc[:,~coffee.columns.duplicated()]

In [5]:
coffee

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,...,unit_price,promo_item_yn,home_store,customer_first-name,customer_email,customer_since,loyalty_card_number,birthdate,gender,birth_year
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,...,2.50,N,3.0,Melissa Johnson,Luke@eget.net,2018-06-19,816-924-9433,1983-02-25,F,1983.0
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,...,3.50,N,3.0,Luke Patel,Herrod@Maecenas.us,2018-11-02,653-218-9979,1991-07-29,N,1991.0
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,...,2.50,N,3.0,Hilel Ballard,Rajah@risus.org,2018-12-30,263-826-9026,1995-02-23,N,1995.0
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,...,2.50,N,3.0,Zephr Zimmerman,Dacey@in.net,2019-03-04,741-320-7166,1999-02-06,F,1999.0
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,...,2.45,N,3.0,Orlando Shields,Ivory@scelerisque.us,2017-10-01,747-164-4596,1967-01-29,M,1967.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49889,753,2019-04-29,16:51:58,8,42,0,N,1,1,30,...,3.00,N,NaN,None,None,None,None,None,None,NaN
49890,756,2019-04-29,16:51:14,8,42,8412,Y,1,1,25,...,2.20,N,8.0,Malcolm,Cedric@neque.us,2019-01-08,193-832-1350,1953-09-16,M,1953.0
49891,759,2019-04-29,11:17:36,8,15,0,Y,1,1,31,...,2.20,N,NaN,None,None,None,None,None,None,NaN
49892,763,2019-04-29,15:45:52,8,45,8030,N,1,1,44,...,2.50,N,8.0,Deirdre,Austin@Nullam.edu,2018-08-23,383-091-4412,1994-01-10,F,1994.0


In [6]:
coffee.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49894 entries, 0 to 49893
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   transaction_id       49894 non-null  int64  
 1   transaction_date     49894 non-null  object 
 2   transaction_time     49894 non-null  object 
 3   sales_outlet_id      49894 non-null  int64  
 4   staff_id             49894 non-null  int64  
 5   customer_id          49894 non-null  int64  
 6   instore_yn           49894 non-null  object 
 7   order                49894 non-null  int64  
 8   line_item_id         49894 non-null  int64  
 9   product_id           49894 non-null  int64  
 10  quantity             49894 non-null  int64  
 11  line_item_amount     49894 non-null  float64
 12  unit_price           49894 non-null  float64
 13  promo_item_yn        49894 non-null  object 
 14  home_store           24852 non-null  float64
 15  customer_first-name  24852 non-null 

In [7]:
coffee.columns

Index(['transaction_id', 'transaction_date', 'transaction_time',
       'sales_outlet_id', 'staff_id', 'customer_id', 'instore_yn', 'order',
       'line_item_id', 'product_id', 'quantity', 'line_item_amount',
       'unit_price', 'promo_item_yn', 'home_store', 'customer_first-name',
       'customer_email', 'customer_since', 'loyalty_card_number', 'birthdate',
       'gender', 'birth_year'],
      dtype='object')

# Data Cleaning - Missing Values

Our info method shows us that each of these columns contain missing values"

* home_store
* customer_first-name 
* customer_email
* customer_since
* loyalty_card_number
* birthdate 
* gender
* birth_year           

This was caused when we joined our sales and customer tables. 

For now we'll fill any missing values with a 0. 

This lets us convert as we need without running into numerical errors.

In [8]:
coffee = coffee.fillna(0)

## Data Cleaning - Column Rename

### Customer -

customer_id	home_store	customer_first-name	customer_email	customer_since	loyalty_card_number	birthdate	gender	birth_year

### Sales

transaction_id	transaction_date	transaction_time	sales_outlet_id	staff_id	customer_id	instore_yn	order	line_item_id	product_id	quantity	line_item_amount	unit_price	promo_item_yn

### Changes 
Column names were shortened or adjusted for clarity.

|Old Name|New Name|
---------------|---------------
|sales_outlet_id|shop_id|
|instore_yn|instore_purchase|
|promo_item_yn'|promo_item|
|customer_first-name|name|
|customer_email|email|
|loyalty_card_number|loyalty_id

In [9]:
coffee.rename(columns={'sales_outlet_id': 'shop_id', 'instore_yn': 'instore_purchase','promo_item_yn': 'promo_item','customer_first-name': 'name',
       'customer_email': 'email', 'loyalty_card_number': 'loyalty_id'}, inplace=True)
coffee.columns

Index(['transaction_id', 'transaction_date', 'transaction_time', 'shop_id',
       'staff_id', 'customer_id', 'instore_purchase', 'order', 'line_item_id',
       'product_id', 'quantity', 'line_item_amount', 'unit_price',
       'promo_item', 'home_store', 'name', 'email', 'customer_since',
       'loyalty_id', 'birthdate', 'gender', 'birth_year'],
      dtype='object')

## Data Cleaning - Y/N to Bool

The instore_purchase column signifies whether a purchase was made **in a store or online**. The value is stored as string of Y for yes or N for no. We will convert this into a **true/false** boolean for ease of analysis.

In [10]:
coffee.loc[coffee["instore_purchase"] == "Y", "instore_purchase"] = True
coffee.loc[coffee["instore_purchase"] == "N", "instore_purchase"] = False
coffee["instore_purchase"]

0        False
1        False
2         True
3        False
4         True
         ...  
49889    False
49890     True
49891     True
49892    False
49893    False
Name: instore_purchase, Length: 49894, dtype: object

## Data Cleaning - Datetime Conversion

We'll convert our time and date columns to datetime objects. This will help us manipulate and plot our time data correctly. Any missing birth_years are set to 1900, being the earliest year recognized by an excel file. This is simple substitution that can be filtered out later.

In [11]:
coffee["transaction_date"] = pd.to_datetime(coffee["transaction_date"])
coffee["transaction_time"] = pd.to_datetime(coffee["transaction_time"])
coffee["birthdate"] = pd.to_datetime(coffee["birthdate"])

# special formating for year
coffee["birth_year"] = coffee["birth_year"].astype(int)
coffee.loc[coffee["birth_year"] == 0, 'birth_year'] = 1900 # placeholder for Missing Values
coffee["birth_year"] = pd.to_datetime(coffee["birth_year"], format='%Y')

# verify
coffee[["transaction_date", "transaction_time", "birthdate", "birth_year"]]

,transaction_date,transaction_time,birthdate,birth_year
0,2019-04-01,2022-06-07 12:04:43,1983-02-25,1983-01-01
1,2019-04-01,2022-06-07 15:54:39,1991-07-29,1991-01-01
2,2019-04-01,2022-06-07 14:34:59,1995-02-23,1995-01-01
3,2019-04-01,2022-06-07 16:06:04,1999-02-06,1999-01-01
4,2019-04-01,2022-06-07 19:18:37,1967-01-29,1967-01-01
...,...,...,...,...
49889,2019-04-29,2022-06-07 16:51:58,1970-01-01,1900-01-01
49890,2019-04-29,2022-06-07 16:51:14,1953-09-16,1953-01-01
49891,2019-04-29,2022-06-07 11:17:36,1970-01-01,1900-01-01
49892,2019-04-29,2022-06-07 15:45:52,1994-01-10,1994-01-01
